# Projeto 1 - Ciência dos Dados

Nome: Guilherme Fugita

Nome: Paola Friedel


Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [17]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[:;\n"'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [18]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\guifu\P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [19]:
filename = 'Big Mac.xlsx'

In [20]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,queria tnt comer um big mac 😫,1
1,"doida pra comer um big mac, daq a pouco vou am...",1
2,vou assistir miss americana dnv enquanto almoç...,1
3,qria um big mac agr,1
4,"@juhrangel_ voltou não, um passarinho me conto...",1


In [21]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,queria tnt comer um big mac 😫,1
1,"doida pra comer um big mac, daq a pouco vou am...",1
2,vou assistir miss americana dnv enquanto almoç...,1
3,qria um big mac agr,1
4,"@juhrangel_ voltou não, um passarinho me conto...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos o produto do McDonalds, o lanche BigMac, tudo que é considerado relevante agrega valores positivos e negativos para o produto como uma critica tanto quanto o desejo dos usuários em relação ao produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [46]:
#--------TUDO--------
with open('txts/tudo.txt', 'w', encoding="utf-8") as todos_tweets:
    for j in range(len(train)):
        tweetTr = train.loc[j,"Treinamento"]
        tweetTrC = cleanup(tweetTr)
        todos_tweets.write("{0} ".format(tweetTrC))
        
#--------RELEVANTES--------
Rel = train.loc[train['Relevância'] == 1, ['Treinamento']]

with open('txts/relevantes.txt', 'w', encoding="utf-8") as tweets_relevantes:
    for q in range(len(Rel)):
        RelTr = Rel.iloc[q, 0]
        RelTrC = cleanup(RelTr)
        tweets_relevantes.write("{0} ".format(RelTrC))

#--------TRRELEVANTES--------
IRel = train.loc[train['Relevância'] == 0, ['Treinamento']]
with open('txts/irrelevantes.txt', 'w', encoding="utf-8") as tweets_irrelevantes:
    for q in range(len(IRel)):
        IRelTr = IRel.iloc[q, 0]
        IRelTrC = cleanup(IRelTr)
        tweets_irrelevantes.write("{0} ".format(IRelTrC))
    
#--------LENDO OS ARQUIVOS--------   
with open("txts/tudo.txt", "r", encoding="utf-8") as arquivo_texto:
    tudo_raw = arquivo_texto.read()  
with open("txts/relevantes.txt", "r", encoding="utf-8") as arquivo_texto:
    relevantes_raw = arquivo_texto.read() 
with open("txts/irrelevantes.txt", "r", encoding="utf-8") as arquivo_texto:
    irrelevantes_raw = arquivo_texto.read() 
  



In [44]:
#--------TUDO--------
todas_palavras = tudo_raw.split()
serie_tudo = pd.Series(todas_palavras_relevantes)
tabela_tudo = serie_tudo.value_counts()
tabela_tudo_relativa = serie_tudo.value_counts(True)

#--------RELEVANTES--------
todas_palavras_relevantes = relevantes_raw.split()
serie_relevantes = pd.Series(todas_palavras_relevantes)
tabela_relevantes = serie_relevantes.value_counts()
tabela_relevantes_relativa = serie_relevantes.value_counts(True)

#--------TRRELEVANTES--------
todas_palavras_irrelevantes = irrelevantes_raw.split()
serie_irrelevantes = pd.Series(todas_palavras_irrelevantes)
tabela_irrelevantes = serie_irrelevantes.value_counts()
tabela_irrelevantes_relativa = serie_irrelevantes.value_counts(True)


In [45]:
probR = len(todas_palavras_relevantes) / len(todas_palavras)
probI = len(todas_palavras_irrelevantes) / len(todas_palavras)

print(round(probR,5))
print(round(probI,5))

0.22389
0.77611


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**